In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss
from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce
import timeit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
import keras
from sklearn.utils import class_weight

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc0

In [ ]:
!pip install autokeras

In [ ]:
import autokeras as ak

## Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv', sep=',')
train = train.set_index('id')

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv', sep=',')
test = test.set_index('id')

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='target',data=train, palette='viridis').set_title('#istances per Class')

In [ ]:
plt.figure(figsize=(10,5))
df= train['target'].value_counts(normalize=True).reset_index()
sns.barplot(x='index',y='target',data=df, palette='viridis').set_title('%istances per Class')

In [ ]:
cols = train.columns[:-1] 
df_unique = pd.concat([train[cols], test[cols]], axis=0)

In [ ]:
plt.figure(figsize=(16,5))
df_unique[cols].nunique().plot(kind='bar', cmap='Spectral', title='Unique values by Feature')

In [ ]:
train_eda = train
train_eda = train_eda.astype('str')
train_eda['Total'] = pd.Series(train_eda[cols].fillna('').values.tolist()).str.join('')

In [ ]:
plt.figure(figsize=(16,5))
train_eda['Total'].value_counts()[:20].plot(kind='bar',title='Duplicated istances')

In [ ]:
cols = train.columns[:-1]
target = train['target']
train2 = train.loc[train_eda.index]

In [ ]:
train = train[cols]

In [ ]:
train['count0'] = (train[cols]==0).sum(axis=1)
train['count1'] = (train[cols]==1).sum(axis=1)
train['count2'] = (train[cols]==2).sum(axis=1)

In [ ]:
train.max()

In [ ]:
tr= train.to_numpy()

In [ ]:
tr.shape

In [ ]:
target2 = pd.get_dummies(target)
target2.head()

## AutoKeras

In [ ]:
model_input = ak.Input()

node1 = ak.Embedding(max_features=360)(model_input)

node5 = ak.ConvBlock()(node1)

node3= ak.RNNBlock()(node5)

node4 = ak.SpatialReduction()(node3)

node2 = ak.DenseBlock()(node4)

output = ak.ClassificationHead()(node2)

In [ ]:
model = ak.AutoModel(inputs=[model_input], outputs=[output], max_trials=5, tuner='bayesian', overwrite=True, loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.015))

In [ ]:
model.fit(x=tr, y=target2, batch_size=64, validation_split=0.25, epochs=150, verbose=1, shuffle=True)

In [ ]:
# get the best model
model1=model.export_model()

In [ ]:
# summarize the loaded model
model1.summary()

In [ ]:
tf.keras.utils.plot_model(
    model1,
    to_file="model1.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True)

In [ ]:
# save the best model
try:
    model1.save("model_autokeras_TPS06", save_format="h5")
except Exception:
    model1.save("model_autokeras.h5")

In [ ]:
test['count0'] = (test[cols]==0).sum(axis=1)
test['count1'] = (test[cols]==1).sum(axis=1)
test['count2'] = (test[cols]==2).sum(axis=1)

In [ ]:
tst = test.to_numpy()

In [ ]:
preds = model1.predict(tst, batch_size=64)

In [ ]:
oof = model1.predict(tr, batch_size=64)

In [ ]:
oof_df = pd.DataFrame(oof, index=train.index)

In [ ]:
predictions = pd.DataFrame(preds, index=test.index)

In [ ]:
predictions.head()

In [ ]:
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv', sep=',')
sub_sample = sub_sample.set_index('id')

In [ ]:
sub_sample['Class_1'] = predictions[0]
sub_sample['Class_2'] = predictions[1]
sub_sample['Class_3'] = predictions[2]
sub_sample['Class_4'] = predictions[3]
sub_sample['Class_5'] = predictions[4]
sub_sample['Class_6'] = predictions[5]
sub_sample['Class_7'] = predictions[6]
sub_sample['Class_8'] = predictions[7]
sub_sample['Class_9'] = predictions[8]

In [ ]:
sub_sample.head()

In [ ]:
sub_sample = sub_sample.reset_index()

In [ ]:
oof_df = oof_df.reset_index()
oof_df.to_csv('oof_df.csv', index=False)

In [ ]:
sub_sample.to_csv('submission.csv',index=False)